In [3]:
import mysql.connector
import pandas as pd
import csv
from mysql.connector import errorcode

In [4]:
mydb = mysql.connector.connect(host = 'localhost', 
                       user = 'root', 
                        )
cursor = mydb.cursor()


In [5]:
DB_NAME = 'diversity'
TABLES = {}

TABLES['parks'] = (
    "CREATE TABLE `parks` ("
    "  `park_code`char(10) NOT NULL,"
    "  `park_name` varchar(50) NOT NULL,"
    "  `state` varchar(10) NOT NULL,"
    "  `acres` int(10) NOT NULL,"
    "  `latitude` double NOT NULL,"
    "  `longitude` double NOT NULL,"
    "  PRIMARY KEY (`park_code`)"#SQL Schema that makes sense and is in at least in first normal form.
    ") ENGINE=InnoDB")
TABLES['species'] = (
    "CREATE TABLE `species` ("
    "  `species_id` varchar(20) NOT NULL,"
    "  `park_name` varchar(50) NOT NULL,"
    "  `category` varchar(20) NOT NULL,"
    "  `order` varchar(20) NOT NULL,"#order is a keyword of the system, so I choose"`"to solve this problem.
    "  `family` varchar(20) NOT NULL,"
    "  `scientific_name` varchar(100) NOT NULL,"
    "  `common_names` text(200) NOT NULL,"
    "  `record_status` enum('Approved','In Review') NOT NULL,"
    "  `occurrence` enum('Not Confirmed','Present','Not Present (False Report)','Not Present (Historical Report)','Not Present') NOT NULL,"
    "  `nativeness` enum('Native','Not Native','Unknown') NOT NULL,"
    "  `abundance` enum('Abundant','Rare','Unknown','Occasional','Common','Uncommon') NOT NULL,"
    "  `seasonality` varchar(50) NOT NULL,"
    "  `conservation_status` varchar(50) NOT NULL,"
    "  PRIMARY KEY (`species_id`)"#SQL Schema that makes sense and is in at least in first normal form.
    ") ENGINE=InnoDB")



In [6]:
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
try:
    mydb.database = DB_NAME  
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        mydb.database = DB_NAME
    else:
        print(err)
        exit(1)
for name, ddl in TABLES.items():
    try:
        print("Creating table {}: ".format(name), end='')
        
        cursor.execute(ddl)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


Creating table parks: OK
Creating table species: OK


In [10]:
csv_data = csv.reader(open('parks.csv'))
first=0
for row in csv_data:
    if first==0:
        first=1
    else:
        cursor.execute('INSERT INTO parks(park_code, \
        park_name, state, acres ,latitude, longitude )'\
        'VALUES(%s, %s,%s,%s,%s,%s)',row)


In [12]:
csv_data = csv.reader(open('species1.csv'))
first=0
for row in csv_data:
    if first==0:
        first=1
    else:
        cursor.execute('INSERT INTO species (species_id, \
        park_name, category,`order`, family, scientific_name ,\
        common_names, record_status, occurrence ,nativeness, abundance ,\
        seasonality, conservation_status)'\
        'VALUES(%s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)',row)
#close the connection to the database.
mydb.commit()
cursor.close()
mydb.close()
print ("Done")

Done
